In [3]:
using Revise
using VariPEPS
using OptimKit
using TensorKit
using JLD2
using Statistics

In [7]:
#= specify the Bond-dimension d, the environment-bond-dimension χ. Further one may specify the model that one wants to test, the 
projectors that should be used in the CTMRG-routine, as well as Hamiltonian parameters. =#
χ = 8
d = 2


#=Here specify the unit cell. For this we define an array "Pattern_arr" that represents the unit cell structure.
The structure is [x-coodinate, y-coordinate]. If two positions within the Pattern_arr are filled with the same number, we assume the tensors at these positions to be identical.
The choice below corresponds to a unit cell like:   A   B 
                                                    B   A=#
Pattern_arr = Array{Any}(undef,2,2)

Pattern_arr[1,1] = 1
Pattern_arr[2,1] = 2
Pattern_arr[1,2] = 3
Pattern_arr[2,2] = 4

#Say more about the possible keywords

keywords = (lattice = :dice, model = :Kitaev_Γ_model_dice, Ham_parameters = Ham_parameters = (h = 0, Φ = 0), Projector_type = :half, Space_type = ℂ, svd_type = :envbond)

(lattice = :dice, model = :Kitaev_Γ_model_dice, Ham_parameters = (h = 0, Φ = 0), Projector_type = :half, Space_type = ℂ, svd_type = :envbond)

In [8]:
#= 
this functiion initializes the PEPS-tensors. 
The first input is the Bond dimension, 
the second input is the dimension of the local Hilbert space, --> This is 2 in the current example, as we are dealing with a spin 1/2 system.
the third one specified how many tensors need to be generated. --> In this example 2 since we have two inequivalent tensors in the unit cell.

What is created here is just an array of TensorMaps with the right number of legs with the desired dimensionality.

Note that we can also enforce local symmetries and only optimize on the parametes for the local tensors that have this symmetry. --> Add mini example later.

adjust this mini-example for all models we have currently: BH, Kitaev(-Gamma), CSL, 
=#
loc_in = initialize_PEPS(d, 8, 4; seed = 1235, lattice = :dice, Number_type = ComplexF64, Space_type = ℂ); #change the code such that if we use real numbers we use "normalization_convention_without_phase"


In [24]:
A = TensorMap(randn, ℂ^2, ℂ^2)
testarr = []
push!(testarr,A)

1-element Vector{Any}:
 TensorMap(ProductSpace(ℂ^2) ← ProductSpace(ℂ^2)):
  0.7341108930736387   0.5418492864063519
 -0.8693658722911339  -2.1229378248052693


In [10]:
#the ctmrg function performs a CTM-RG procedure. -- Put more details on elementwise convergence and so on.
#= note that in some cases, the GKL algorithm, that is used for the SVD in the calculation of the projectors finds invariant subspaces and fails. 
In these cases the code will fall back to conventional SVD algorithms. If one does not want to use the GKL algorithm at all, we can use the keyword svd_type = :envbond
to just use conventional SVD algorithms.=#

#=In the "energy_and_gradient" function as well as in the "ctmrg" function we have a keyword "adjust_χ" which is by default set to false. In that case the 
function works normally. However if we do not set the keyword to false but give instead a tuple, during the CTMRG routine the algorithm automatically increases the bond dimension
up to a maximal value "χ_max". This happens whenever the truncation error "err_trunc" that we obtain in the creation of the projectors is larger than a choosen tolerance err_tol.
Thus we set the keyword to "adjust_χ = (err_tol, χ_max)" in case we want the bond dimension to be increased during the optimization.=#

ctmrg_res = ctmrg(loc_in, χ, Pattern_arr; keywords..., observ = true, conv_info = false, maxiter = 200, adjust_χ = false)

(0.005419056017902424, Any[-0.029518015582136025 + 8.460676473560518e-19im, -0.029518015582136025 + 8.460676473560518e-19im, -0.029518015582136025 + 8.460676473560518e-19im, 0.03148381130859058 + 5.288017721120368e-19im, -0.06387294969199274 - 1.0858866565180646e-17im, 0.015520200657184995 - 4.2008867988758584e-18im, -0.03404456485224787 - 2.1273097268191742e-17im, 0.003993932011804158 + 1.895609450930599e-17im, -0.0778735624032803 + 7.434344617931986e-18im, -0.02561773131184254 - 9.590309084430813e-18im  …  0.03197661013875813 + 6.258437148027391e-18im, 0.026921220760898746 - 1.4817868842973693e-17im, 0.026921220760898746 - 1.4817868842973693e-17im, 0.026921220760898746 - 1.4817868842973693e-17im, -0.03053242980505658 + 7.245505350333841e-18im, 0.006617806012931028 + 5.983630958197442e-18im, -0.030418337061694018 + 4.511461080266696e-18im, -0.04640442955989809 - 2.428210096774647e-17im, 0.018150584897241723 + 4.58072008163038e-18im, 0.08763804416212088 + 2.8232445007151156e-17im])

For a state, obtained after a few optimization steps of the Heisenberg model we look at the truncation errors as 
a function of the bond dimension - with the corresponding energy. The upper block is for the "fullfishman"-projectors
while the lower block is for the "half"-projectors. We conclude from the convergence of the energy-expectation values
that a maximal trunctation error of ~10^-3 is a reasonable starting point. It can always be chosen more stricly.

E = -0.6603737287187401 @ Bond_env = 80, maximum(trunc_sv_arr) = 5.975686042268443e-5 
E = -0.6603737358938596 @ Bond_env = 60, maximum(trunc_sv_arr) = 0.00012481759922763688
E = -0.6603737799922393 @ Bond_env = 40, maximum(trunc_sv_arr) = 0.00031194042937930866
E = -0.6603736187795066 @ Bond_env = 30, maximum(trunc_sv_arr) = 0.0006405720791402293
E = -0.6603724597506875 @ Bond_env = 26, maximum(trunc_sv_arr) = 0.0009362616891090525
E = -0.6603639207050201 @ Bond_env = 20, maximum(trunc_sv_arr) = 0.0020002907780533145
E = -0.6603793732337668 @ Bond_env = 16, maximum(trunc_sv_arr) = 0.003927764975811235


E = -0.6603737141782002 @ Bond_env = 80, maximum(trunc_sv_arr) = 8.859010589321488e-5
E = -0.6603736039935315 @ Bond_env = 60, maximum(trunc_sv_arr) = 0.00018689030517925998
E = -0.6603739415374238 @ Bond_env = 40, maximum(trunc_sv_arr) = 0.0004921465970557896
E = -0.6603749442733124 @ Bond_env = 30, maximum(trunc_sv_arr) = 0.0011953646129798726
E = -0.6603675868543281 @ Bond_env = 26, maximum(trunc_sv_arr) = 0.00172234383475717
E = -0.6603589442727148 @ Bond_env = 20, maximum(trunc_sv_arr) = 0.003446092986088426
E = -0.6603566068572576 @ Bond_env = 16, maximum(trunc_sv_arr) = 0.005807880961672133

In [11]:
#the function energy_and_gradient is needed for the variational optimization. It calculates the gradient at the fixed point of the CTM-RG iteration.
#= note that in some cases, the GKL algorithm, that is used for the SVD in the calculation of the projectors finds invariant subspaces and fails. 
In these cases the code will fall back to conventional SVD algorithms. If one does not want to use the GKL algorithm at all, we can use the keyword svd_type = :envbond
to just use conventional SVD algorithms.=#

#=In the "energy_and_gradient" function as well as in the "ctmrg" function we have a keyword "adjust_χ" which is by default set to false. In that case the 
function works normally. However if we do not set the keyword to false but give instead a tuple, during the CTMRG routine the algorithm automatically increases the bond dimension
up to a maximal value "χ_max". This happens whenever the truncation error "err_trunc" that we obtain in the creation of the projectors is larger than a choosen tolerance err_tol.
Thus we set the keyword to "adjust_χ = (err_tol, χ_max)" in case we want the bond dimension to be increased during the optimization.=#

e, gr = energy_and_gradient(loc_in, χ, Pattern_arr; keywords...)

Info: it took 8 CTMRG steps to converge the SV to 1.0e-6
Info: it took 10 CTMRG steps to converge the environments element wise to 1e-6


(0.005419056034296243, TrivialTensorMap{ComplexSpace, 2, 3, Matrix{ComplexF64}}[TensorMap((ℂ^2 ⊗ ℂ^2) ← (ℂ^2 ⊗ ℂ^2 ⊗ (ℂ^8)')):
[:, :, 1, 1, 1] =
 0.0030498165969155954 + 0.0017147198699128793im  …  -0.001547201081360283 + 0.00022838454259034398im
 0.0014405490745132534 + 0.0027773975793281016im      -0.00241362406385083 + 0.001975509641189415im

[:, :, 2, 1, 1] =
  0.001478684742839195 + 0.0022778877589605586im  …   -0.0005485008638864933 + 0.0014991606734365975im
 0.0020865709185055935 - 0.0010375910117869738im     -0.00036685394158595487 - 0.004267360239694745im

[:, :, 1, 2, 1] =
  0.005310300181052463 + 0.001029480314990574im  …  0.004830508465267399 + 0.00262365583816656im
 -0.005778970000765664 - 0.008447063308704492im     0.003509307875945003 - 0.002283549387523031im

[:, :, 2, 2, 1] =
 -0.002036444744864111 + 0.002184557384611209im   …  -0.005900490054029326 + 0.0008658653685276286im
 0.0045310461237253845 + 0.0039016858140106946im       0.00286187357321175 + 0.0016642205311478

In [12]:
#for the optimization we need the following two functions:

function _finalize!(x,f,g, numiter)
    flush(stdout)
    flush(stderr)

    
    #=if numiter%2 == 1
        file = jldopen("PEPS_during_optimization.jld2", "a+")
        write(file, "PEPS_at_opt_step_$(numiter)", x)
        close(file)
    end=#
     
    
    return x, f, g
end

function _inner(x, ξ_1, ξ_2)
    return dot(real(ξ_1), real(ξ_2)) + dot(imag(ξ_1), imag(ξ_2))
end

_inner (generic function with 1 method)

In [13]:
#we specify the parameters that are used in the optimization.

optimparas1 = (gradtol = 1e-4, verbosity = 2, maxiter = 100, linesearch = HagerZhangLineSearch(; c₁ = 1e-4, c₂ = 0.9, maxiter = 40))
kwargs = (finalize! = _finalize!, inner = _inner)


(finalize! = _finalize!, inner = _inner)

In [15]:

#χ_arr represents the starting bond dimension. It is being increased according to the accuracy demands set by the parameters in keyword "adjust_χ"

x, e, gr, numfg, normgradhistory = optimize(x -> energy_and_gradient(x, χ, Pattern_arr; keywords..., adjust_χ = false), loc_in, LBFGS(15; optimparas1... ); kwargs... )

Info: it took 8 CTMRG steps to converge the SV to 1.0e-6
Info: it took 10 CTMRG steps to converge the environments element wise to 1e-6


┌ Info: LBFGS: initializing with f = 0.005419056034, ‖∇f‖ = 1.0270e-01
└ @ OptimKit /Users/weerde/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:35


Info: it took 9 CTMRG steps to converge the SV to 1.0e-6
Info: it took 11 CTMRG steps to converge the environments element wise to 1e-6


┌ Info: LBFGS: iter    1: f = -0.185307691290, ‖∇f‖ = 9.6775e-02, α = 1.00e+00, m = 0, nfg = 1
└ @ OptimKit /Users/weerde/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:76


Info: it took 13 CTMRG steps to converge the SV to 1.0e-6
Info: it took 16 CTMRG steps to converge the environments element wise to 1e-6


...

Info: it took 5 CTMRG steps to converge the SV to 1.0e-6
Info: it took 9 CTMRG steps to converge the environments element wise to 1e-6


┌ Info: LBFGS: iter   61: f = -0.669580274064, ‖∇f‖ = 1.0457e-04, α = 1.00e+00, m = 15, nfg = 1
└ @ OptimKit /Users/weerde/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:76


Info: it took 5 CTMRG steps to converge the SV to 1.0e-6
Info: it took 9 CTMRG steps to converge the environments element wise to 1e-6


┌ Info: LBFGS: converged after 62 iterations: f = -0.669603226875, ‖∇f‖ = 7.7687e-05
└ @ OptimKit /Users/weerde/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:138


(TrivialTensorMap{ComplexSpace, 2, 3, Matrix{ComplexF64}}[TensorMap((ℂ^2 ⊗ ℂ^2) ← (ℂ^2 ⊗ ℂ^2 ⊗ (ℂ^8)')):
[:, :, 1, 1, 1] =
 -1.2054102639222335 - 1.2756775325204623im  …   0.5581802376461252 + 0.35040820775983944im
   3.358479045824903 + 0.9658852484720937im     -1.7105365715429552 + 0.16068044175832996im

[:, :, 2, 1, 1] =
 -1.1487523820097445 + 0.8053725885211518im  …  0.22226760941622636 - 0.5741737841720816im
 -1.9482206329750869 + 2.2564888704660397im     0.30888264798360027 - 1.1669806793956434im

[:, :, 1, 2, 1] =
 -0.43900165650376355 + 1.4281460541805822im  …  -0.03655406194815737 - 0.6594711586334114im
   -0.729386120837561 - 3.511994620924595im        0.9126820128748567 + 1.5522154778093251im

[:, :, 2, 2, 1] =
 1.392123219834655 + 0.6008991104758742im  …  -0.8495400105014898 + 0.10586363592485304im
 2.956307063362705 + 0.3062208757314907im     -1.2115630249907658 + 0.4591099525274614im

[:, :, 1, 1, 2] =
 0.9873948919966431 + 0.17028599782666204im  …  -0.5730853051034702 + 